In [1]:
import openai 
from dotenv import load_dotenv, find_dotenv
import json
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from typing import Optional
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool
from langchain_community.document_loaders import UnstructuredURLLoader

import os

def read_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data

dotenv_path = os.path.join('..', '.env')
load_dotenv(dotenv_path)
openai.api_key = os.environ['OPENAI_API_KEY']
root_dir = '../db/raw/20240802/'
job_data = read_json(os.path.join(root_dir,'job_process_list_sample.json'))

In [2]:
job_data[0]['query_list'][0]['name']

'Microsoft'

In [3]:
search = GoogleSearchAPIWrapper()


def top5_results(query):
    return search.results(query, 5)


tool = Tool(
    name="Google Search Snippets",
    description="Search Google for recent results.",
    func=top5_results,
)

a = tool.run("site:linkedin.com What is company Equip")


/journel/s0/zur74/venv/gpt_rag/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GoogleSearchAPIWrapper`.
  warn_deprecated(


In [36]:
import requests
from bs4 import BeautifulSoup
import json

def scrape_website(url):
    # Send a GET request to the URL
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Initialize a dictionary to store the scraped data
    data = {
        "url": url,
        "title": soup.title.string if soup.title else "",
        "meta_description": "",
        "headings": [],
        "paragraphs": [],
        "links": []
    }
    
    # Extract meta description
    meta_desc = soup.find("meta", attrs={"name": "description"})
    if meta_desc:
        data["meta_description"] = meta_desc.get("content", "")
    
    # Extract headings
    for heading in soup.find_all(['h1', 'h2', 'h3']):
        data["headings"].append({
            "level": heading.name,
            "text": heading.text.strip()
        })
    
    # Extract paragraphs
    for para in soup.find_all('p'):
        data["paragraphs"].append(para.text.strip())
    
    # Extract links
    for link in soup.find_all('a'):
        data["links"].append({
            "text": link.text.strip(),
            "href": link.get("href", "")
        })
    
    return data

# Use the function
url = 'https://equip.health/'
url = 'https://www.linkedin.com/company/equip-behavioral-health/'
structured_data = scrape_website(url)
input_data = json.dumps(structured_data)

In [37]:
from langchain_community.document_loaders import UnstructuredURLLoader
def add_url(original_url):
    original_url += '/about/'
    return original_url

urls = [add_url(a[0]['link'])]

loader =  UnstructuredURLLoader(urls=urls)
documents = loader.load()[0]

class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the company this url is describing.")
    industry: str = Field(description="which industry this company is working for")
    size: int = Field(description="Size of this company - number of empolyees")
    founding_year: int = Field(description="Year this company was founded")
    company_url: str = Field(description="URL of the company's website")
    funding: str = Field(description="Funding status of the company - how much do they recieve from investor - which series")

overview_extraction_function = [
    convert_pydantic_to_openai_function(Overview)
]

model = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages([
    ("system",
        "extract information from web page query results. Your goal is to \
    get informatino about company. If you can't find relevant information, \
    please return None"
    ),
    ("human", "{input}")
])

tagging_model = model.bind(
    functions=overview_extraction_function,
    function_call={"name":"Overview"}
)

tagging_chain = prompt | tagging_model

chain_result = tagging_chain.invoke({"input": input_data})

json_response = json.loads(
    chain_result.additional_kwargs['function_call']['arguments']
    )



In [38]:
json_response

{'summary': 'Equip provides families with lasting recovery from eating disorders through dedicated virtual care teams and gold-standard treatment delivered at home.',
 'industry': 'Mental Health Care',
 'size': 490,
 'founding_year': 0,
 'company_url': 'https://equip.health/',
 'funding': 'US$ 35.0M in Last Round, Series C'}

In [17]:
from langchain.document_loaders import UnstructuredURLLoader

# Correct way: Pass a list of URLs
loader = UnstructuredURLLoader(urls=['https://www.linkedin.com/company/equip-behavioral-health/'])
documents = loader.load()

In [19]:
documents[0].page_content

''

In [13]:
job_data[2]['query_list'][2]

{'name': 'Equip',
 'position': {'name': 'Machine Learning (ML) Engineer',
  'date': 'Thu, 1 Aug 2024',
  'location': 'United States',
  'apply_link': 'https://www.linkedin.com/comm/jobs/view/3971448321/?trackingId=EQPRHmEGAqjNXjC3Y1zz4Q%3D%3D&refId=ByteString%28length%3D16%2Cbytes%3Decf8c8b7...62180f8e%29&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3BuS11K6yeRlGY9Y6WRhpwpQ%3D%3D&midToken=AQGuxmwVoa47_g&midSig=1V2vtRM4GC4bo1&trk=eml-email_job_alert_digest_01-job_card-0-view_job&trkEmail=eml-email_job_alert_digest_01-job_card-0-view_job-null-blvbqx~lzbowikn~gb-null-null&eid=blvbqx-lzbowikn-gb&otpToken=MTUwNTFmZTgxNDJmY2RjN2I3MjQwNGVkNDQxY2UyYjA4ZWM2ZDU0NTk5YWI4NjYxNzljMDA1NmI0NzVmNWVmNmY2ZDNkZjliNzdjZWMxODI3ZThiZDZiN2NjZGM0N2IzOThmOTNmMmRmODIxNTdhYjlkYTc4YSwxLDE%3D',
  'description': None}}

In [15]:
email_data[2]

{'email_id': '190dc2edb4112e4b',
 'metadata': {'subject': '“data scientist”: TikTok - Data Scientist - Machine Learning and more',
  'sender': 'LinkedIn Job Alerts <jobalerts-noreply@linkedin.com>',
  'date': 'Mon, 22 Jul 2024 20:42:52 +0000 (UTC)',
  'recipients': ['Zihan Ren <zihanren.ds@gmail.com>'],
  'cc': [''],
  'bcc': ['']},
 'content': {'body': 'Your job alert for data scientist in California\r\n        \r\n30+ new jobs match your preferences.  \r\n          \r\nData Scientist - Machine Learning\r\nTikTok\r\nSan Jose, CA\r\nFast growing\r\nView job: https://www.linkedin.com/comm/jobs/view/3981983527/?trackingId=wFGQhzzGoSUCu7UymFD0mg%3D%3D&refId=ByteString%28length%3D16%2Cbytes%3Def1a3709...168c7c88%29&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_digest_01%3Bhro8Zef6TnaIIE84lxg9zg%3D%3D&midToken=AQGuxmwVoa47_g&midSig=38uwfJo3SqSbk1&trk=eml-email_job_alert_digest_01-job_card-0-view_job&trkEmail=eml-email_job_alert_digest_01-job_card-0-view_job-null-blvbqx~lyxgcqpy~br-null-null&